# Extracting Structured Data from Websites

- This notebook demonstrates the process of extracting structured data from PDFs and websites using custom tools and models. We'll use various tools from the CrewAI library to perform the extraction, define a data schema with Pydantic, and set up an agent to handle the extraction task.
- You need to add BrowserBase API key (BROWSERBASE_API_KEY) project ID (BROWSERBASE_PROJECT_ID) and  in your .env so that the agent can access the webpage.


## Installing Required Libraries

Before proceeding with the extraction, we need to install necessary libraries such as `crewai`, `pydantic`, and `crewai_tools`. These libraries provide the functionalities needed for building the extraction tools and parsing the outputs.


In [1]:
#!pip install crewai pydantic crewai_tools browserbase

In [2]:
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser
from crewai import Task, Agent
from crewai_tools import PDFSearchTool, FileReadTool, BrowserbaseLoadTool
from crewai import Crew, Process

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


## Defining Data Models

We define a Pydantic model `CompanyQuarterlyReport` to represent the structured data that we aim to extract. This model includes fields for company name, fiscal year, quarter, revenue, and other relevant data points.


In [3]:
from typing import Optional, List, Literal

class CompanyQuarterlyReport(BaseModel):
    company_name: str = Field(..., description="Name of the company")
    fiscal_year: int = Field(..., description="Fiscal year of the report")
    quarter: Literal['Q1', 'Q2', 'Q3', 'Q4'] = Field(..., description="Fiscal quarter (e.g., Q1, Q2, Q3, Q4)")
    quarter_revenue: float = Field(..., description="Revenue for the quarter in USD")
    yoy_quarter_revenue_growth: Optional[float] = Field(None, description="Year-over-year revenue growth for the quarter in percentage")
    key_feature_updates: str = Field(None, description="Key feature updates released during the quarter")
    summary: str = Field(None, description="Summarize the quarterly report in one paragraph")

## Output Parsing Instructions

To ensure the extracted data conforms to our defined schema, we use `PydanticOutputParser` to generate the necessary output format instructions. This ensures that our extraction process adheres to the expected schema.


In [4]:
pr_schema_parser = PydanticOutputParser(pydantic_object=CompanyQuarterlyReport)
print(pr_schema_parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"company_name": {"description": "Name of the company", "title": "Company Name", "type": "string"}, "fiscal_year": {"description": "Fiscal year of the report", "title": "Fiscal Year", "type": "integer"}, "quarter": {"description": "Fiscal quarter (e.g., Q1, Q2, Q3, Q4)", "enum": ["Q1", "Q2", "Q3", "Q4"], "title": "Quarter", "type": "string"}, "quarter_revenue": {"description": "Revenue for the quarter in USD", "title": "Quarter Revenue", "type": "number"}, "yoy_quarter_revenue_growth": {"anyOf": [{"type": "number"}, {"type": "nu

In [5]:
import os
with open(os.path.join('/mnt/data/', 'press_release_schema_format_instructions.txt'), 'w') as f:
    f.write(pr_schema_parser.get_format_instructions())

## Setting Up Extraction Tools and Agent

We initialize tools such as `PDFSearchTool`, `FileReadTool`, and `BrowserbaseLoadTool` to handle different aspects of the extraction process, such as searching PDFs and reading files. An agent is set up to coordinate the extraction tasks.


In [6]:
extract_prompt = """
=====================
   TASK OVERVIEW
=====================

You have access to the following paths:

- **Source Website Link**: `{website}`
- **Extraction/formatting instruction**: `{formatting_instruction}`

Your task is to extract data from the press release article located at `{website}` and structure it into the JSON format following the instructions in `{formatting_instruction}`.

=====================
   INSTRUCTIONS
=====================

1. **Extract Key Data**:
   - Extract key data points like company name, fiscal year, quarter, quarter revenue, YOY quarter revenue growth, key feature updates, summary, as outlined in `{formatting_instruction}`.

--------------------------------------------------

2. **Accuracy Check**:
   - Cross-verify the extracted data with the website and ensure all relevant fields are extracted.
   - Mark missing or unreadable data as 'N/A' or 'Not Processable'.

--------------------------------------------------

3. **Output Data**:
   - Structure and pass the data for validation.

===========================
   END OF INSTRUCTIONS
===========================
"""

In [7]:
pdf_tool = PDFSearchTool()
txt_read_tool = FileReadTool()
browser_tool = BrowserbaseLoadTool()
tools = [pdf_tool, txt_read_tool, ]

In [8]:
extracting_agent = Agent(
            role='Senior Data Analyst',
            goal='Extract data specified in the {formatting_instruction} from the source {website}.',
            backstory=(
                "You are a detailed-oriented data analyst. You have strong analytical skills that allow you to identify and "
                "abstract analytical concepts. You are familiar with different data formats such as YAML, JSON and work well "
                "with software engineers.You are proud of your attention to details and will triple check the results for "
                "accuracy together with your coworkers."
            ),
            tools=tools,
        )

In [9]:
extraction_task = Task(
            description=extract_prompt,
            expected_output='Extracted data points in JSON format for press release.',
            tools=tools,
            agent=extracting_agent,
            output_json=CompanyQuarterlyReport,
            # human_input=True
        )


## Creating and Executing the Task

We create a task to extract the required data based on the given instructions. The task is executed by the agent, and the output is validated against our data model.


In [10]:
crew = Crew(
    agents=[
        extracting_agent,
    ],
    tasks=[
        extraction_task,
    ],
    process= Process.sequential, 
    memory=True,
    cache=True,
    max_rpm=100,
    output_json = CompanyQuarterlyReport,
    output_log_file = 'extractor.log'
)

## Running the Extraction Process

The `Crew` object is initialized to manage the agents and tasks in a sequential manner. The extraction process is run with specific settings such as memory usage, caching, and rate limits.


In [11]:
crew_output = crew.kickoff(
    inputs={
        'website': 'https://seekingalpha.com/pr/19805912-apple-reports-third-quarter-results',
        'formatting_instruction': '/mnt/data/press_release_schema_format_instructions.txt',
})

Inserting batches in chromadb: 100% 1/1 [00:00<00:00,  2.01it/s]
Inserting batches in chromadb: 100% 1/1 [00:00<00:00,  3.34it/s]
Inserting batches in chromadb: 100% 1/1 [00:00<00:00,  3.12it/s]


 Error parsing JSON: Expecting value: line 1 column 1 (char 0). Attempting to handle partial JSON.
 Pydantic validation error: 1 validation error for CompanyQuarterlyReport
  Invalid JSON: key must be a string at line 5 column 30 [type=json_invalid, input_value='{\n  "company_name": "Ap... Placeholder summary\n}', input_type=str]
    For further information visit https://errors.pydantic.dev/2.8/v/json_invalid. The JSON structure doesn't match the expected model. Attempting alternative conversion method.


Inserting batches in chromadb: 100% 1/1 [00:00<00:00,  2.03it/s]


## Conclusion and Results

After running the extraction process, the extracted data is analyzed for accuracy and completeness. Further steps may include refining the extraction logic or improving the data model based on the results.


In [12]:
import json
crew_output.json_dict

{'company_name': 'Apple Inc.',
 'fiscal_year': 2023,
 'quarter': 'Q3',
 'quarter_revenue': 81400.0,
 'yoy_quarter_revenue_growth': 2.0,
 'key_feature_updates': 'Released new MacBook Pro, announced iOS 16, expanded Apple Pay services',
 'summary': 'Apple Inc. reported its financial results for the third quarter of the fiscal year 2023, showing a revenue of $81.4 billion with a year-over-year growth of 2%. The quarter saw the release of the new MacBook Pro, the announcement of iOS 16, and the expansion of Apple Pay services.'}